# A*

In [136]:
class Node:
    
    def __init__(self,value,coord):
        self.value=value
        self.coord=coord
        self.parent=None
        self.g=0
        self.h=0
        
    def __str__(self):
        s=f'{self.coord} f={self.g+self.h:0.2f} , g={self.g:0.2f}, h={self.h:0.2f}'
        return s
    
    def move_cost(self,other):
        return 1

In [137]:
def children(current,grid):
    x,y=current.coord
    links=[(x-1,y),(x,y-1),(x,y+1),(x+1,y)]
    
    valid_link= [link for row in grid for link in row if link.value!=0]
    valid_children=[link for link in valid_link if link.coord in links]
    return valid_children

In [138]:
def manhattan(node, goal):
    xN,yN=node.coord
    xG,yG=goal.coord
    h=abs(xN-xG)+abs(yN-yG)
    return h

In [139]:
def astar(start , goal , grid):
    current= start
    openl=list()
    closedl=list()
    
    openl.append(current)
    
    while openl:
        current= min(openl, key=lambda o:o.g+o.h)
        
        if current==goal:
            path=[]
            while current.parent:
                path.append(current)
                current=current.parent
            path.append(current)
            return path[::-1]
        
        openl.remove(current)
        closedl.append(current)
        
        for node in children(current,grid):
            
            if node in closedl:
                new_cost=current.g+current.move_cost(node)
                if new_cost<=node.g:
                    openl.append(node)
                    closedl.remove(node)
                    
            elif node in openl:
                new_cost=current.g+current.move_cost(node)
                if new_cost<=node.g:
                    node.g=new_cost
                    node.parent=current
                    
            else:
                node.g=current.g+current.move_cost(node)
                node.h=manhattan(node,goal)
                node.parent=current
                openl.append(node)
                
    return None

In [140]:
# grid =[[1,1,0,0,0,1,1],
#        [1,1,1,1,0,1,1],
#        [1,1,1,1,0,1,1],
#        [1,1,1,1,0,1,1],
#        [1,1,0,0,0,1,1],
#        [1,1,1,1,1,1,1],
#        [1,1,1,1,1,1,1] ]

grid = [[1,1,1,1],
        [1,1,1,1],
        [1,1,1,1],
        [1,1,0,0],
        [1,1,0,1]]

for x in range(len(grid)):
    for y in range(len(grid[x])):
        grid[x][y]=Node(grid[x][y],(x,y))
        
# start=grid[5][1]
# goal=grid[1][3]

start = grid[4][0]
goal = grid [0][3]

In [141]:
path= astar(start,goal,grid)
if path:
    for i in path:
        print(i.coord,end=" ")
else:
    print("No path")

(4, 0) (4, 1) (3, 1) (2, 1) (2, 2) (2, 3) (1, 3) (0, 3) 

# ANN

In [142]:
import numpy as np
x=np.array(([2,9],[1,5],[3,6]),dtype='float')
y=np.array(([92],[86],[89]),dtype='float')
x=x/np.amax(x,axis=0)
actual_output=y/100

In [143]:
lr=0.15
epochs=1000
inp_layer=2
out_layer=1
hid_layer=3

wh=np.random.uniform(low=-0.05,high=0.05,size=(inp_layer,hid_layer))
bh=np.random.uniform(low=-0.05,high=0.05,size=(1,hid_layer))
wout=np.random.uniform(low=-0.05,high=0.05,size=(hid_layer,out_layer))
bout=np.random.uniform(low=-0.05,high=0.05,size=(1,out_layer))

In [144]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def der_sigmoid(x):
    return x*(1-x)

In [145]:
for i in range(epochs):
    hinp=np.dot(x,wh)+bh
    hlayer_act=sigmoid(hinp)
    
    oinp=np.dot(hlayer_act,wout)+bout
    output=sigmoid(oinp)
    
    hgrad=der_sigmoid(hlayer_act)
    ograd=der_sigmoid(output)
    
    EO=actual_output-output
    d_output=EO*ograd
    
    EH=d_output.dot(wout.T)
    d_hiddenlayer=EH*hgrad
    
    wh+=x.T.dot(d_hiddenlayer)*lr
    wout+=hlayer_act.T.dot(d_output)*lr

In [146]:
print("Input",str(x))
print("ActualOutput",str(actual_output))
print("PredictedOutput",str(output))

Input [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
ActualOutput [[0.92]
 [0.86]
 [0.89]]
PredictedOutput [[0.89595784]
 [0.87911769]
 [0.8948187 ]]


# ID3

In [147]:
import numpy as np
import pandas as pd
import math

data=pd.read_csv('PlayTennis.csv')
data

,Outlook,Temperature,Humidity,Wind,PlayTennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [148]:
features=[feat for feat in data]
features.remove("PlayTennis")
features

['Outlook', 'Temperature', 'Humidity', 'Wind']

In [149]:
class Node:
    def __init__(self):
        self.value=""
        self.children=[]
        self.isLeaf=False

In [150]:
def entropy(examples):
    pc=0.0
    nc=0.0
    for _,row in examples.iterrows():
        if row["PlayTennis"]=="Yes":
            pc+=1
        else:
            nc+=1
    if pc==0.0 or nc==0.0:
        return 0.0
    else:
        p=pc/(pc+nc)
        n=nc/(pc+nc)
        return -(p*math.log(p,2)+n*math.log(n,2))

In [151]:
def info_gain(examples,attr):
    gain=entropy(examples)
    uniq=np.unique(examples[attr])
    
    for u in uniq:
        subdata=examples[examples[attr]==u]
        sub_e=entropy(subdata)
        gain-=(float(len(subdata))/float(len(examples)))*sub_e
        
    return gain

In [152]:
def ID3(examples, attr):
    max_gain=0.0
    max_feat=""
    
    root=Node()
    
    for feature in attr:
        gain=info_gain(examples,feature)
        if gain>=max_gain:
            max_gain=gain
            max_feat=feature
    root.value= max_feat
    
    uniq=np.unique(examples[max_feat])
    
    for u in uniq:
        subdata=examples[examples[max_feat]==u]
        
        if entropy(subdata)==0.0:
            newNode=Node()
            newNode.value=u
            newNode.isLeaf=True
            newNode.pred=np.unique(subdata["PlayTennis"])
            root.children.append(newNode)
        else:
            dummyNode=Node()
            dummyNode.value=u
            new_attr=attr.copy()
            new_attr.remove(max_feat)
            child=ID3(subdata,new_attr)
            dummyNode.children.append(child)
            root.children.append(dummyNode)
            
    return root

In [153]:
def printTree(root:Node, depth=0):
    for i in range(depth):
        print('\t',end="")
    print(root.value,end="")
    if root.isLeaf:
        print("->",root.pred)
    print()
    for child in root.children:
        printTree(child,depth+1)

In [154]:
root=ID3(data,features)
printTree(root)

Outlook
	Overcast-> ['Yes']

	Rain
		Wind
			Strong-> ['No']

			Weak-> ['Yes']

	Sunny
		Humidity
			High-> ['No']

			Normal-> ['Yes']



In [155]:
def classify(root: Node,new):
    for child in root.children:
        if child.value==new[root.value]:
            if child.isLeaf:
                print("Predicted label for",new,"is:",child.pred)
                exit
            else:
                classify(child.children[0],new)

In [156]:
new = {"Outlook":"Sunny", "Temperature":"Hot", "Humidity":"Normal", "Wind":"Strong"}
classify (root, new)

Predicted label for {'Outlook': 'Sunny', 'Temperature': 'Hot', 'Humidity': 'Normal', 'Wind': 'Strong'} is: ['Yes']


# Naive Bayes

In [214]:
filename="diabetes.csv"
import pandas as pd
import numpy as np
import math
df=pd.read_csv(filename)
df=df.astype(float)
train=df.sample(frac=0.8, random_state=100)
test=df.drop(train.index)

In [215]:
outcome_group=train.groupby(df.columns[-1])
n_attr=len(df.columns)-1
summaries={}

for classValue,instances in outcome_group:
    attr_mv=[]
    mean=list(instances.mean(axis=0).values)
    std_dev=list(instances.std(axis=0).values)
    for i in range(n_attr):
        attr_mv.append([mean[i],std_dev[i]])
    summaries[classValue]=attr_mv

In [216]:
def calculateProb(x,mean,stddev):
    expval=math.exp(-math.pow((x-mean),2)/(2*math.pow(stddev,2)))
    return (1/(math.sqrt(2*math.pi*math.pow(stddev,2))))*expval

In [217]:
def calculateClassProb(summaries,x_vec):
    probabilities={}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue]=1
        for i in range(len(classSummaries)):
            mean,stddev=classSummaries[i]
            x=x_vec[i]
            probabilities[classValue]*=calculateProb(x,mean,stddev)
    return probabilities

In [218]:
def predict(summaries,x_vec):
    prob=calculateClassProb(summaries,x_vec)
    bestLabel,bestProb=None,-1
    
    for classValues, probability in prob.items():
        if bestLabel is None or probability>bestProb:
            bestProb=probability
            bestLabel=classValue
    return bestLabel

In [219]:
predictions=[]
testSet=test.values.tolist()

for i in range(len(testSet)):
    result= predict(summaries,testSet[i])
    predictions.append(result)

In [220]:
def getAccuracy(test, predictions):
    correct = 0
    for i in range(len(test)):
        if test.iloc[i,-1] == predictions[i]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

accuracy = getAccuracy(test, predictions)
print(f'Split {len(df)} rows into train={len(train)} and test={len(test)}')
print(f'Accuracy: {accuracy}')

Split 768 rows into train=614 and test=154
Accuracy: 34.41558441558442
